In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import pickle

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder
import xgboost as xgb

In [ ]:
raw_df = pd.read_csv('heart.csv')

In [ ]:
raw_df.shape

In [ ]:
raw_df.columns = raw_df.columns.str.lower()

In [ ]:
raw_df.columns 

In [ ]:
raw_df.isnull().sum()

In [ ]:
raw_df.heartdisease.value_counts(normalize=True)

In [ ]:
raw_df.heartdisease.mean()

In [ ]:
raw_df.nunique()

In [ ]:
raw_df.age.value_counts()

# Data Scaling 

In [ ]:

mms = MinMaxScaler() # Normalization
ss = StandardScaler() # Standardization

raw_df['oldpeak'] = mms.fit_transform(raw_df[['oldpeak']])
raw_df['age'] = ss.fit_transform(raw_df[['age']])
raw_df['restingbp'] = ss.fit_transform(raw_df[['restingbp']])
raw_df['cholesterol'] = ss.fit_transform(raw_df[['cholesterol']])
raw_df['maxhr'] = ss.fit_transform(raw_df[['maxhr']])
raw_df.head()

# Feature Analysis 

In [ ]:
df_full_train, df_test = train_test_split(raw_df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [ ]:
len(df_train), len(df_val), len(df_test)

In [ ]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
y_train = df_train.heartdisease .values
y_val = df_val.heartdisease.values
y_test = df_test.heartdisease.values

del df_train['heartdisease']
del df_val['heartdisease']
del df_test['heartdisease']

In [ ]:
df_full_train.head(5)

In [ ]:
df_full_train.heartdisease.value_counts()

In [ ]:
mutual_col = list(df_train.columns)
mutual_col

# Convert qualitative data to quantitative

In [ ]:
categorical_columns = list(df_full_train.dtypes[df_full_train.dtypes == 'object'].index)
categorical_columns

In [ ]:
numerical_columns = list(df_full_train.dtypes[df_full_train.dtypes != 'object'].index)
numerical_columns

In [ ]:
le = LabelEncoder() 
df_full_train[categorical_columns] = df_full_train[categorical_columns].apply(lambda col: le.fit_transform(col)) 
df_full_train.head(5)

In [ ]:
mutual_scores = []
for c in  df_full_train[mutual_col].columns: 
    score = round(mutual_info_score(df_full_train.heartdisease,df_full_train[c]),3)
    mutual_scores.append(score)
    print(f"mutual score for {c} is {score}")

In [ ]:
import matplotlib.pyplot as plt

sorted_mutual_scores, sorted_mutual_col_names = zip(*sorted(zip(mutual_scores, mutual_col)))
plt.bar(sorted_mutual_col_names, sorted_mutual_scores)
plt.xlabel("Features")
plt.ylabel("Mutual Information Scores")
plt.title("Mutual Information Scores for Features")
plt.xticks(rotation='vertical')
plt.show()

# Feature importance: Heartdisease rate and risk ratio

In [ ]:
for column in categorical_columns:
    unique_categories = df_full_train[column].unique() # this is important
    print(f"Heart disease rate for {column}:")
    for category in unique_categories:
        mean = df_full_train[df_full_train[column] == category].heartdisease.mean()
        print(f"{category}: {mean}")
    print("\n")

# Risk Ratio

In [ ]:
global_heartdisease = df_full_train.heartdisease.mean()

In [ ]:
for c in categorical_columns:
    print(c)
    df_group = df_full_train.groupby(c).heartdisease.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] -global_heartdisease
    df_group['risk'] = df_group['mean'] /global_heartdisease
    df_group
    print(df_group)
    print()

# Feature importance:Correlation

In [ ]:
numeric_columns = list(df_full_train.dtypes[df_full_train.dtypes != 'object'].index)
numeric_columns

In [ ]:
data_numeric = df_full_train[numeric_columns]
data_numeric.describe()

In [ ]:
data_numeric.corr()

In [ ]:
plt.figure(figsize=(9, 6))
sns.heatmap(data_numeric.corr())
plt.title('Heatmap showing correlations between numerical data')
plt.show();

In [ ]:
threshold = 0.1


columns_to_drop = [col for col in df_full_train.columns if mutual_info_score(df_full_train.heartdisease,df_full_train[col])< threshold]


df_full_train = df_full_train.drop(columns=columns_to_drop)
df_full_train

In [ ]:
df_full_train, df_test = train_test_split(raw_df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.heartdisease .values
y_val = df_val.heartdisease.values
y_test = df_test.heartdisease.values

del df_train['heartdisease']
del df_val['heartdisease']
del df_test['heartdisease']

# One Hot Encoding

In [ ]:
dv = DictVectorizer(sparse=False)

train_dict = df_train.to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val.to_dict(orient='records')
X_val = dv.transform(val_dict)

# Logistic Regression

In [ ]:
model = LogisticRegression(solver='lbfgs')
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]

In [ ]:
heartdisease_decision = (y_pred >= 0.5)

In [ ]:
(y_val == heartdisease_decision).mean()

In [ ]:
df_pred = pd.DataFrame()
df_pred['probability'] = y_pred
df_pred['prediction'] = heartdisease_decision.astype(int)
df_pred['actual'] = y_val
df_pred

# Logistic Regression Score

In [ ]:
roc_auc_score(y_val, y_pred)

# Decision Tree 

In [ ]:
df_full_train, df_test = train_test_split(raw_df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.heartdisease .values
y_val = df_val.heartdisease.values
y_test = df_test.heartdisease.values

del df_train['heartdisease']
del df_val['heartdisease']
del df_test['heartdisease']

In [ ]:
scores = []
depths = [4,5,6]

for depth in depths: 
    for s in [1, 5, 10, 15, 20, 500, 100, 200]:
        x_dict = df_train.to_dict(orient='records')
        dv= DictVectorizer(sparse=False)
        x_train = dv.fit_transform(x_dict)
        dt = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=s)
        dt.fit(x_train, y_train)
        val_dicts = df_val.to_dict(orient='records')
        x_val = dv.transform(val_dicts)
        y_pred = dt.predict_proba(x_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        scores.append((depth, s, auc))

In [ ]:
columns = ['max_depth', 'min_samples_leaf', 'auc']
df_scores = pd.DataFrame(scores, columns=columns)

In [ ]:
df_scores_pivot = df_scores.pivot(index='min_samples_leaf', columns=['max_depth'], values=['auc'])
df_scores_pivot.round(3)

In [ ]:
sns.heatmap(df_scores_pivot, fmt=".3f");

In [ ]:
dt = DecisionTreeClassifier(max_depth=5, min_samples_leaf=5)
dt.fit(X_train, y_train)

# Decision Tree Score

In [ ]:
val_dicts = df_val.to_dict(orient='records')
x_val = dv.transform(val_dicts)
y_pred = dt.predict_proba(x_val)[:, 1]
auc = roc_auc_score(y_val, y_pred)
auc

# Random Forest Classifier

In [ ]:
df_full_train, df_test = train_test_split(raw_df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.heartdisease .values
y_val = df_val.heartdisease.values
y_test = df_test.heartdisease.values

del df_train['heartdisease']
del df_val['heartdisease']
del df_test['heartdisease']

In [ ]:
scores = []

for d in [5, 10, 15]:
    for n in range(10, 201, 10):
        rf = RandomForestClassifier(n_estimators=n,
                                    max_depth=d,
                                    random_state=1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        scores.append((d, n, auc))

In [ ]:
columns = ['max_depth', 'n_estimators', 'auc']
df_scores = pd.DataFrame(scores, columns=columns)

In [ ]:
df_scores_pivot = df_scores.pivot(index='n_estimators', columns=['max_depth'], values=['auc'])
df_scores_pivot.round(3)

In [ ]:
sns.heatmap(df_scores_pivot, fmt=".3f");

In [ ]:
for d in [5, 10, 15]:
    df_subset = df_scores[df_scores.max_depth == d]
    
    plt.plot(df_subset.n_estimators, df_subset.auc,
             label='max_depth=%d' % d)

plt.legend();

In [ ]:
max_depth = 15

In [ ]:
scores = []

for s in [1, 3, 5, 10, 50]:
    for n in range(10, 201, 10):
        rf = RandomForestClassifier(n_estimators=n,
                                    max_depth=max_depth,
                                    min_samples_leaf=s,
                                    random_state=1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        scores.append((s, n, auc))

In [ ]:
columns = ['min_samples_leaf', 'n_estimators', 'auc']
df_scores = pd.DataFrame(scores, columns=columns)

In [ ]:
sns.heatmap(df_scores_pivot, fmt=".3f");

In [ ]:
colors = ['black', 'blue', 'orange', 'red', 'grey']
values = [1, 3, 5, 10, 50]

for s, col in zip(values, colors):
    df_subset = df_scores[df_scores.min_samples_leaf == s]
    
    plt.plot(df_subset.n_estimators, df_subset.auc,
             color=col,
             label='min_samples_leaf=%d' % s)

plt.legend();

In [ ]:
min_samples_leaf = 3

In [ ]:
scores = []
for n in range(10, 201, 10):
        rf = RandomForestClassifier(n_estimators=n,
                                    max_depth=max_depth,
                                    min_samples_leaf=min_samples_leaf,
                                    random_state=1)
        rf.fit(X_train, y_train)

        y_pred = rf.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_pred)

        scores.append((s, n, auc))

In [ ]:
columns = ['max_depth', 'min_samples_leaf', 'auc']
df_scores = pd.DataFrame(scores, columns=columns)

In [ ]:
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=max_depth,
                            min_samples_leaf=min_samples_leaf,random_state=1)

rf.fit(x_train, y_train)

# Random Forest Score

In [ ]:
y_pred = rf.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred)
auc

# Gradient boosting and XGBoost

In [ ]:
df_full_train, df_test = train_test_split(raw_df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
y_train = df_train.heartdisease .values
y_val = df_val.heartdisease.values
y_test = df_test.heartdisease.values

del df_train['heartdisease']
del df_val['heartdisease']
del df_test['heartdisease']

In [ ]:
x_dict = df_train.to_dict(orient='records')
dv= DictVectorizer(sparse=False)
x_train = dv.fit_transform(x_dict)
val_dicts = df_val.to_dict(orient='records')
x_val = dv.transform(val_dicts)

In [ ]:
features = dv.get_feature_names_out()
features

In [ ]:
features = dv.get_feature_names_out()
features = features.tolist()
dtrain = xgb.DMatrix(x_train, label=y_train, feature_names=features)
dval = xgb.DMatrix(x_val, label=y_val, feature_names=features)

In [ ]:
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=10)

In [ ]:
y_pred = model.predict(dval)
roc_auc_score(y_val, y_pred)

In [ ]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [ ]:
%%capture output

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=10,
                  verbose_eval=5,
                  evals=watchlist)
y_pred = model.predict(dval)

# Gradient boosting and XGBoost Score

In [ ]:
roc_auc_score(y_val, y_pred)

In [ ]:
s = output.stdout

In [ ]:
print(s[:200])

In [ ]:
def parse(output):
    scores = []
    for line in output.stdout.strip().split('\n'):
            a,b,c = line.split('\t')
            itr = int(a.strip('[]'))
            train = float(b.split(':')[1])
            val = float(c.split(':')[1])
            
            scores.append((itr,train,val))
    columns = ['iteration','train_auc','val_auc']
    df_scores = pd.DataFrame(scores,columns=columns)
    return df_scores

In [ ]:
df_score = parse(output)

In [ ]:
plt.plot(df_score.iteration, df_score.train_auc, label='train')
plt.plot(df_score.iteration, df_score.val_auc, label='val')
plt.legend();

In [ ]:
def final_train(df_train, y_train, C=1.0):
    dicts = df_train.to_dict(orient='records')

    dv = DictVectorizer(sparse=False)
    x_train = dv.fit_transform(dicts)

    model =LogisticRegression(solver='lbfgs')
    model.fit(x_train, y_train)
    
    return dv, model


def final_predict(df_test, dv, model):
    dicts = df_test.to_dict(orient='records')

    x_test = dv.transform(dicts)
    y_pred = model.predict_proba(x_test)[:, 1]

    return y_pred

In [ ]:
dv, model = final_train(df_full_train, df_full_train.heartdisease.values, C=1.0)
y_pred = final_predict(df_test, dv, model)

In [ ]:
dv, model,y_pred

In [ ]:
output_file =  'mid_term_model'

In [ ]:
with open(output_file,'wb')as f_out:
    pickle.dump((dv,model),f_out)
print(f'the model is saved to {output_file}')